In [1]:
from graph_tool import *
from tqdm import tqdm
import csv

In [2]:
sample_celltype_dict = dict()
with open('../data/sampleToCelltype.csv') as sampleToCelltype:
    reader = csv.reader(sampleToCelltype)
    for row in reader:
        if row[0] != "sampleID":
            sample_celltype_dict[row[0]] = [row[1], row[2]]

# sample_celltype_dict

In [4]:
ID_celltype_dict = dict()
with open('../data/sampleToCelltype.csv') as sampleToCelltype:
    reader = csv.reader(sampleToCelltype)
    for row in reader:
        if row[0] != "sampleID":
            ID_celltype_dict[row[1]] = row[2]

            
# ID_celltype_dict

In [4]:
# rem_dict{celltypes : {samples: {REMs: score}}}
# dict to map celltype to samples and the included REMs with their scores

rem_dict = dict()
with open('../data/REMActivity_1.csv') as file_1:
    reader_1 = csv.reader(file_1)
    for r in tqdm(reader_1):
        rem = r[0] 
        sample = r[1] 
        score = r[3] 
        
        if sample_celltype_dict.get(sample) == None:       # if sample not 
            continue
        else:
            celltype = sample_celltype_dict.get(sample)[0]
            
        if rem_dict.get(celltype) == None:
            rem_dict[celltype] = {sample: {rem: score}}
            
        elif rem_dict[celltype].get(sample) == None:
            rem_dict.get(celltype)[sample] = {rem: score}
            
        else:
            rem_dict[celltype].get(sample)[rem] = score
            

399206927it [16:16, 408751.21it/s]


In [20]:
# create template graph to fill with celltype specific values afterwards
# Graph contains gens and rems and their edges buth with no values

g = Graph()
name = g.new_vp("string")
g.vp.name = name
type = g.new_vp("string")
g.vp.type = type
celltypeID = g.new_ep("string")
g.edge_properties["celltypeID"] = celltypeID
celltype = g.new_ep("string")
g.edge_properties["celltype"] = celltype

done_proteins_dict = {}
with open('../data/REMAnnotationModelScore_1.csv') as file:
    reader = csv.reader(file)
    for row in tqdm(reader):
        rem = row[4]
        gen = row[3]
        if row[1] != "start":
            v1 = g.add_vertex()
            g.vp.name[v1] = rem
            g.vp.type[v1] = "REM"

            if done_proteins_dict.get(gen) == None:
                v2 = g.add_vertex()
                g.vp.name[v2] = gen
                g.vp.type[v2] = "Gene"
                done_proteins_dict[gen] = int(v2)
            else:
                v2 = g.vertex(done_proteins_dict.get(gen))

            e = g.add_edge(v1, v2)
                
g.save(f"../data/graph-tool/template.gt")

2404862it [00:24, 98214.93it/s] 


In [ ]:
# load template graph and fill with celltype edge values by iterating over samples in celltype

graph = load_graph(f"../data/graph-tool/template.gt")

for celltype_id in rem_dict:
#     if celltype_id in ["CTID_0000006", "CTID_0000027", "CTID_0000033"]:
#         continue
    print(celltype_id)
    
    g = Graph(graph)
    samples = rem_dict.get(celltype_id)
    celltype_name = ID_celltype_dict.get(celltype_id)
    
    for sample in samples:
        sampleID = g.new_ep("double")
        g.edge_properties[sample] = sampleID
    
        for e in tqdm(g.edges()):
            rem = g.vp.name[e.source()]
            g.ep.celltypeID[e] = celltype_id
            g.ep.celltype[e] = celltype_name
        
#         for sample in samples:
#             sampleID = g.new_ep("double")
#             g.edge_properties[sample] = sampleID
            g.ep[sample][e] = samples.get(sample).get(rem)
            
    g.save(f"../data/graph-tool/{celltype_id}.gt")
#     break

In [ ]:
with open('../data/REMActivity_1.csv') as file:
    reader = csv.reader(file)
    counter = 0
    for row in tqdm(reader):
        print(row)
        counter += 1
        if(counter == 200):
            break

In [ ]:
with open('../data/REMAnnotationModelScore_1.csv') as file:
    reader = csv.reader(file)
    counter = 0
    for row in tqdm(reader):
        print(row)
        counter += 1
        if(counter == 200):
            break

In [ ]:
# check if all edges are the same in all samples
d = dict()
with open('../data/REMActivity_1.csv') as file:
    reader = csv.reader(file)
    counter = 0
    for row in tqdm(reader):
        if row[0] != "REMID":
            if d.get(row[1]) != None:
                d[row[1]].add(row[0])
            else:
                d[row[1]] = {row[0]}
        
check = list(d.values())[0]
for k in d.keys():
#     print(k)
    if  len(check.difference(d.get(k))) != 0:
        print(k)

In [ ]:
for celltype_id in rem_dict:
    print(celltype_id)
    
    g = Graph()
    name = g.new_vp("string")
    g.vp.name = name
    type = g.new_vp("string")
    g.vp.type = type
    celltypeID = g.new_ep("string")
    g.edge_properties["celltypeID"] = celltypeID
    celltype = g.new_ep("string")
    g.edge_properties["celltype"] = celltype
    
#         standDnase1Log2 = g.new_ep("double")
#         g.edge_properties["standDnase1Log2"] = standDnase1Log2

    done_proteins_dict = {}
    with open('../data/REMAnnotationModelScore_1.csv') as file:
        reader = csv.reader(file)
        for row in tqdm(reader):
            rem = row[4]
            gen = row[3]
            if row[1] != "start":
                v1 = g.add_vertex()
                g.vp.name[v1] = rem
                g.vp.type[v1] = "REM"

                if done_proteins_dict.get(gen) == None:
                    v2 = g.add_vertex()
                    g.vp.name[v2] = gen
                    g.vp.type[v2] = "Gene"
                    done_proteins_dict[gen] = int(v2)
                else:
                    v2 = g.vertex(done_proteins_dict.get(gen))

                e = g.add_edge(v1, v2)
#                     g.ep.standDnase1Log2[e] = rem_dict.get(rem)[1]
                g.ep.celltypeID[e] = celltype_id
                g.ep.celltype[e] = ID_celltype_dict.get(celltype_id)
                
                samples = rem_dict.get(celltype_id)
                for sample in samples:
                    sampleID = g.new_ep("double")
                    g.edge_properties[sample] = sampleID
                    g.ep[sample][e] = samples.get(sample).get(rem)
                
    g.save(f"../data/graph-tool/{celltype_id}.gt")
